- 스파크에는 여섯 가지 '핵심' 데이터소스와 커뮤니티에서 만든 수백 가지 외부 데이터 소스가 있음
    - 다양한 데이터소스를 읽고 쓰는 능력과 데이터 소스를 커뮤니티에서 자체적으로 만들어내는 능력은 스파크를 가장 강력하게 만들어주는 힘
- 스파크의 핵심 데이터소스
    - CSV
    - JSON
    - 파케이
    - ORC
    - JDBC/ODBC 연결
    - 일반 텍스트파일
- 커뮤니티에서 만든 수많은 데이터 소스 중 일부
    - 카산드라
    - HBase
    - 몽고디비
    - AWS Redshift
    - XML
    - 기타 수많은 데이터소스

# 9.1 데이터소스 API의 구조

## 9.1.1 읽기 API 구조
- 데이터 읽기의 핵심 구조
    DataFrameReader.format(...).option("key", "value").schema(...).load()
- 모든 데이터소스를 읽을 때 위와 같은 형식 사용
    - format 메서드는 선택적으로 사용할 수 있으며, 기본값은 파케이 포맷
    - option 메서드를 사용해 데이터를 읽는 방법에 대한 파라미터를 키-값 쌍으로 설정
    - schema 메서드는 데이터 소스에서 스키마를 제공하거나, 스키마 추론 기능을 사용하려는 경우에 선택적으로 사용할 수 있음
    - 데이터 포맷별로 필요한 몇 가지 옵션이 존재

## 9.1.2 데이터 읽기의 기초
- 스파크에서 데이터를 읽을 때는 기본적으로 DataFrameReader를 사용
- DataFrameReader는 SparkSession의 read 속성으로 접근
    Spark.read
- DataFrameReader를 얻은 다음에는 다음과 같은 값을 지정
    - 포맷
    - 스키마
    - 읽기 모드
    - 옵션
- 포맷, 스키마, 그리고 옵션은 트랜스포메이션을 추가로 정의할 수 있는 DataFrameReader를 반환
- 그리고 읽기 모드를 제외한 세가지 항목은 필요한 경우에만 선택적으로 지정할 수 있음
- 데이터소스마다 데이터를 읽는 방식을 결정할 수 있는 옵션 제공
- 사용자는 DataFrameReader에 반드시 데이터를 읽을 경로를 지정해야 함
### 읽기 모드
- 스파크가 형식에 맞지 않는 데이터를 만났을 때의 동작 방식을 지정하는 옵션
    - Permissive : 오류 레코드의 모든 필드를 null로 설정하고 모든 오류 레코드를 \_corrupt_record라는 문자열 컬럼에 기록
    - dropMalformed : 형식에 맞지 않는 레코드가 포함된 로우 제거
    - failFast : 형식에 맞지 않는 레코드를 만나면 즉시 종료
- 읽기 모드의 기본값은 Permissive

## 9.1.3 쓰기 API 구조
- 데이터 쓰기의 핵심 구조
    DataFrameWriter.format(...).option(...).partitonBy(...).bucketBy(...).sortBy(...).save()
- format 메서드는 선택적으로 사용할 수 있으며 기본값은 파케이 포맷
- option 메서드를 사용해 데이터 쓰기 방법을 설정

## 9.1.4 데이터 쓰기의 기초
- 데이터 쓰기는 데이터 읽기와 매우 유사
    - DataFrameReader 대신 DataFrameWriter를 사용


### 저장 모드
- 저장 모드는 스파크가 지정된 위치에서 동일한 파일이 발견했을 때의 동작 방식을 지정하는 옵션
    - append : 해당 경로에 이미 존재하는 파일 목록에 결과 파일을 추가
    - overwrite : 이미 존재하는 모든 데이터를 완전히 덮어 씀
    - errorIfExists : 해당 경로에 데이터나 파일이 존재하는 경우 오류를 발생시키면서 쓰기 작업 실패
    - ignore : 해당 경로에 데이터나 파일이 존재하는 경우 아무런 처리도 하지 않음
- 기본값은 errorIfExists

# 9.2 CSV 파일
- comma-separated values. 콤마(,)로 구분된 값
- 각 줄이 단일 레코드가 되며 레코드의 각 필드를 콤마로 구분하는 일반적인 텍스트 파일 포맷

## 9.2.1 CSV 옵션
- 교재 pg 250-251

## 9.2.2 CSV 파일 읽기

In [2]:
spark

res1: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@44409a29


In [22]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}


In [23]:
val myManualSchem = new StructType(Array(
    new StructField("DEST_COUNTRY_NAME", StringType, true),
    new StructField("ORIGIN_COUNTRY_NAME", StringType, true),
    new StructField("count", LongType, false)
    ))

myManualSchem: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,false))


In [24]:
spark.read.format("csv")
.option("header", "true")
.option("mode", "FAILFAST")
.schema(myManualSchem)
.load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv")
.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [25]:
val myManualSchema = new StructType(Array(
    new StructField("DEST_COUNTRY_NAME", LongType, true),
    new StructField("ORIGIN_COUNTRY_NAME", LongType, true),
    new StructField("count", LongType, false)
    ))

myManualSchema: org.apache.spark.sql.types.StructType = StructType(StructField(DEST_COUNTRY_NAME,LongType,true),StructField(ORIGIN_COUNTRY_NAME,LongType,true),StructField(count,LongType,false))


In [26]:
spark.read.format("csv")
.option("header", "true")
.option("mode", "FAILFAST")
.schema(myManualSchema)
.load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv")
.take(5)

23/03/21 21:36:01 ERROR Executor: Exception in task 0.0 in stage 12.0 (TID 19)
org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file file:///Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv. Details: 
	at org.apache.spark.sql.errors.QueryExecutionErrors$.cannotReadFilesError(QueryExecutionErrors.scala:731)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:283)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:364)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:890)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at o

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 19) (192.168.0.2 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Encountered error while reading file file:///Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv. Details:

## 9.2.3 CSV 파일 쓰기

In [27]:
val csvFile = spark.read.format("csv")
.option("header", "true").option("mode", "FAILFAST").schema(myManualSchem)
.load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv")

csvFile: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [28]:
csvFile.write.format("csv").mode("overwrite").option("sep","\t")
.save("/tmp/my-tsv-file.tsv")

# 9.3 JSON 파일

## 9.3.1 JSON 옵션
- 교재 pg 255-256

## 9.3.2 JSON 파일 읽기


In [29]:
spark.read.format("json").option("mode", "FAILFAST").schema(myManualSchem)
.load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/json/2010-summary.json").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



## 9.3.3 JSON 파일 쓰기
- 파티션당 하나의 파일을 만들며 전체 DataFrame을 단일 폴더에 저장
- JSON 객체는 한 줄에 하나씩 기록됨

In [30]:
csvFile.write.format("json").mode("overwrite").save("/tmp/my-json-file.json")

# 9.4 파케이 파일
- 컬럼 기반의 데이터 저장 방식
- 분석 워크로드에 최적화
- 저장소 공간 절약, 전체 파일을 읽는 대신 개별 컬럼을 읽을 수 있으며, 컬럼 기반의 압축 기능 제공
- 아파치 스파크와 잘 호환되기 때뭉네 스파크의 기본 파일 포맷
- 읽기 연산 시 JSON이나 CSV보다 훨씬 효율적으로 동작하므로 장기 저장용 데이터는 파케이 포맷으로 저장하는 것이 좋음
- 복합 데이터 타입을 지원
    - 컬럼이 배열, 맵, 구조체 데이터 타입이라 해도 문제없이 읽고 쓸 수 있음
    - csv에서는 배열 사용 불가


## 9.4.1 파케이 파일 읽기
- 파케이는 옵션이 거의 없음
    - 데이터를 저장할 때 자체 스키마를 사용해 데이터를 저장하기 때문
    - 포맷을 설정하는 것만으로 충분
- DataFrame을 표현하기 위해 정확한 스키마가 필요한 경우에만 스키마를 설정

In [31]:
spark.read.format("parquet")

res15: org.apache.spark.sql.DataFrameReader = org.apache.spark.sql.DataFrameReader@2ee04fbc


In [32]:
spark.read.format("parquet").load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/parquet/2010-summary.parquet").show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



### 파케이 옵션
- 교재 pg 259

## 9.4.2 파케이 파일 쓰기

In [33]:
csvFile.write.format("parquet").mode("overwrite").save("/tmp/my-parquet-file.parquet")

# 9.5 ORC 파일
- ORC는 하둡 워크로드를 위해 셜계된 자기 기술적이며 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷
- 이 포맷은 대규모 스트리밍 읽기에 최적화되어 있을 뿐만 아니라 필요한 로우를 신속하게 찾아낼 수 있는 기능이 통합되어 있음
- 스파크는 ORC 파일 포맷을 효율적으로 사용할 수 있으므로 별도의 옵션 지정 없이 데이터를 읽을 수 있음
- 파케이는 스파크에 최적화된 반면 ORC는 하이브에 최적화

## 9.5.1 ORC 파일 읽기

In [34]:
spark.read.format("orc").load("../../../Downloads/Spark-The-Definitive-Guide-master/data/flight-data/orc/2010-summary.orc")

res18: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


## 9.5.2 ORC 파일 쓰기

In [35]:
csvFile.write.format("orc").mode("overwrite").save("/tmp/my-orc-file.orc")

# 9.6 SQL 데이터베이스
- SQL 데이터소스는 매우 강력한 커넥터 중 하나
    - 사용자는 SQL을 지원하는 다양한 시스템에 SQL 데이터소스를 연결
    - e.g. MySQL, PostgreSQL, Oracle 데이터베이스에 접속, SQLite에 접속
- 데이터베이스는 원시 파일 형태가 아니므로 고려해야 할 옵션이 더 많음
    - e.g. 데이터베이스의 인증 정보나 접속과 관련된 옵션 필요
    - 스파크 클러스터에서 데이터베이스 시스템에 접속 가능한지 네트워크 상태 확인 필요
- 데이터베이스를 설정하는 번거로움을 없애고 이 책의 목적에 충실하기 위해 SQLite 실행을 위한 참고용 샘플 제공

### JDBC 데이터소스 옵션
- 교재 pg 262-263 참고

## 9.6.1 SQL 데이터베이스 읽기
- 파일 읽기와 마찬가지로 SQL 데이터베이스에서 데이터를 읽는 방법은 다른 데이터소스와 다르지 않음

In [3]:
val driver = "org.sqlite.JDBC"
val path = "/Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/jdbc/my-sqlite.db"
val url = s"jdbc:sqlite:/${path}"
val tablename = "flight_info"

driver: String = org.sqlite.JDBC
path: String = /Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/jdbc/my-sqlite.db
url: String = jdbc:sqlite://Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/jdbc/my-sqlite.db
tablename: String = flight_info


- 접속 관련 속성을 정의한 다음, 정상적으로 데이터베이스에 접속되는지 테스트해 해당 연결이 유효한지 확인 가능

In [4]:
val dbDataFrame = spark.read.format("jdbc").option("url", url)
.option("dbtable", tablename).option("driver", driver).load()

dbDataFrame: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [5]:
dbDataFrame.select("DEST_COUNTRY_NAME").distinct().show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|         Anguilla|
|           Russia|
|         Paraguay|
|          Senegal|
|           Sweden|
+-----------------+
only showing top 5 rows



## 9.6.2 쿼리 푸시다운
- 스파크는 DataFrame을 만들기 전에 데이터베이스 자체에서 데이터를 필터링하도록 만들 수 있음
- 스파크는 특정 유형의 쿼리를 더 나은 방식으로 처리할 수 있음
    - 예를 들어, DataFrame에 필터를 명시하면 스파크는 해당 필터에 대한 처리를 데이터베이스로 위임(push down)

In [7]:
dbDataFrame.filter("DEST_COUNTRY_NAME in ('Anuguilla', 'Sweden')").explain

== Physical Plan ==
*(1) Scan JDBCRelation(flight_info) [numPartitions=1] [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2] PushedFilters: [*In(DEST_COUNTRY_NAME, [Anuguilla,Sweden])], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:decimal(20,0)>




In [8]:
val pushdownQuery = """(SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info) AS flight_info"""

pushdownQuery: String = (SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info) AS flight_info


In [9]:
val dbDataFrame = spark.read.format("jdbc")
.option("url", url).option("dbtable", pushdownQuery).option("driver", driver)
.load()

dbDataFrame: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string]


In [10]:
dbDataFrame.explain()

== Physical Plan ==
*(1) Scan JDBCRelation((SELECT DISTINCT(DEST_COUNTRY_NAME) FROM flight_info) AS flight_info) [numPartitions=1] [DEST_COUNTRY_NAME#12] PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>




### 데이터베이스 병렬로 읽기
- 스파크는 파일 크기, 파일 유형 그리고 압축 방식에 따른 '분할 가능성'에 따라 여러 파일을 읽어 하나의 파티션으로 만들거나 여러 파티션을 하나의 파일로 만드는 기본 알고리즘을 가지고 있음
    - 파일이 가진 이런 유연성은 SQL 데이터베이스에도 존재하지만 몇 가지 수동 설정이 필요
    - numPartitions 옵션을 사용해 읽기 및 쓰기용 동시 작업 수를 제한할 수 있는 최대 파티션 수를 설정할 수 있음

In [11]:
val dbDataFrame = spark.read.format("jdbc")
.option("url", url).option("dbtable", tablename).option("driver", driver)
.option("numPartitions", 10).load()

dbDataFrame: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [12]:
val props = new java.util.Properties
props.setProperty("driver", "org.sqlite.JDBC")

props: java.util.Properties = {driver=org.sqlite.JDBC}
res4: Object = null


In [13]:
val predicates = Array(
    "DEST_COUNTRY_NAME = 'Sweden' OR ORIGIN_COUNTRY_NAME = 'Sweden'",
    "DEST_COUNTRY_NAME = 'Anguilla' or ORIGIN_COUNTRY_NAME = 'Anguilla'")

predicates: Array[String] = Array(DEST_COUNTRY_NAME = 'Sweden' OR ORIGIN_COUNTRY_NAME = 'Sweden', DEST_COUNTRY_NAME = 'Anguilla' or ORIGIN_COUNTRY_NAME = 'Anguilla')


In [14]:
spark.read.jdbc(url, tablename, predicates, props).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|           Sweden|      United States|   65|
|    United States|             Sweden|   73|
|         Anguilla|      United States|   21|
|    United States|           Anguilla|   20|
+-----------------+-------------------+-----+



- 연관성 없는 조건절을 정의하면 중복 로우가 많이 발생할 수 있음

In [15]:
val props = new java.util.Properties
props.setProperty("driver", "org.sqlite.JDBC")

props: java.util.Properties = {driver=org.sqlite.JDBC}
res6: Object = null


In [16]:
val predicates = Array(
    "DEST_COUNTRY_NAME != 'Sweden' OR ORIGIN_COUNTRY_NAME != 'Sweden'",
    "DEST_COUNTRY_NAME != 'Anguilla' OR ORIGIN_COUNTRY_NAME != 'Anguilla'")

predicates: Array[String] = Array(DEST_COUNTRY_NAME != 'Sweden' OR ORIGIN_COUNTRY_NAME != 'Sweden', DEST_COUNTRY_NAME != 'Anguilla' OR ORIGIN_COUNTRY_NAME != 'Anguilla')


In [17]:
spark.read.jdbc(url, tablename, predicates, props).count()

res7: Long = 510


### 슬라이딩 윈도우 기반의 파티셔닝

In [18]:
val colName = "count"
val lowerBound = 0L
val upperBound = 348113L
val numPartitions = 10

colName: String = count
lowerBound: Long = 0
upperBound: Long = 348113
numPartitions: Int = 10


In [19]:
spark.read.jdbc(url, tablename, colName, lowerBound, upperBound, numPartitions, props)
.count()

res8: Long = 255


## 9.6.3 SQL 데이터베이스 쓰기
- URI를 지정하고 지정한 쓰기 모드에 따라 데이터를 쓰면 됨

In [36]:
val newPath = "jdbc:sqlite://tmp/my-sqlite.db"
csvFile.write.mode("overwrite").jdbc(newPath, tablename, props)

23/03/21 21:36:23 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


newPath: String = jdbc:sqlite://tmp/my-sqlite.db


In [37]:
spark.read.jdbc(newPath, tablename, props).count()

res21: Long = 255


In [38]:
csvFile.write.mode("append").jdbc(newPath, tablename, props)

23/03/21 21:37:03 WARN JdbcUtils: Requested isolation level 1 is not supported; falling back to default isolation level 8


# 9.7 텍스트 파일
- 스파크는 일반 텍스트 파일도 읽을 수 있음
    - 파일의 각 줄은 DataFrame의 각 레코드가 됨

## 9.7.1 텍스트 파일 읽기
- textfile 메서드에 텍스트 파일을 지정하기만 하면 됨
    - textfile 메서드는 파티션 수행 결과로 만들어진 디렉터리명을 무시
    - 파티션된 텍스트 파일을 읽거나 쓰려면 읽기 및 쓰기 시 파티션 수행 결과로 만들어진 디렉터리를 인식할 수 있도록 text 메서드를 사용해야 함

spark.read.textFile("/Users/choyubin/Downloads/Spark-The-Definitive-Guide-master/data/flight-data/csv/2010-summary.csv")
.selectExpr("split(value ',') as rows").show()

## 9.7.2 텍스트 파일 쓰기
- 텍스트 파일을 쓸 때는 문자열 컬럼이 하나만 존재해야 함

In [40]:
csvFile.select("DEST_COUNTRY_NAME").write.text("/tmp/simple-text-file.txt")

In [41]:
csvFile.limit(10).select("DEST_COUNTRY_NAME","count")
.write.partitionBy("count").text("/tmp/five-csv-files2.csv")

# 9.8 고급 I/O 개념
- 쓰기 작업 전에 파티션 수를 조절함으로써 병렬로 처리할 파일 수를 제어할 수 있음
    - 버켓팅
    - 파티셔닝


## 9.8.1 분할 가능한 파일 타입과 압축 방식
- 특정 파일 포맷은 기본적으로 분할을 지원
- 압축 방식 관리 필요
    - 모든 압축 방식이 분할 압축을 지원하지는 않음
    - 데이터를 저장하는 방식에 따라 스파크 잡이 원활하게 동작하는 데 막대한 영향을 끼칠 수 있음
    - 추천하는 파일 포맷과 압축 방식은 파케이 파일 포맷과 GZIP 압축 방식


## 9.8.2 병렬로 데이터 읽기
- 여러 익스큐터가 같은 파일을 동시에 읽을 수는 없지만 여러 파일을 동시에 읽을 수는 있음
    - 다수의 파일이 존재하는 폴더를 읽을 때 폴더의 개별 파일은 DataFrame의 파티션이 됨
    - 따라서 사용 가능한 익스큐터를 이용해 병렬(익스큐터 수를 넘어가는 파일은 처리 중인 파일이 완료될 때까지 대기)로 파일을 읽기


## 9.8.3 병렬로 데이터 쓰기
- 파일이나 데이터 수는 데이터를 쓰는 시점에 DataFrame이 가진 파티션 수에 따라 달라질 수 있음
- 기본적으로 데이터 파티션 당 하나의 파일이 작성됨
    - 따라서 옵션에 지정된 파일명은 실제로는 다수의 파일을 가진 디렉터리
    - 그리고 디렉터리 안에 파티션당 하나의 파일로 데이터를 저장

In [42]:
csvFile.repartition(5).write.format("csv").save("/tmp/multiple.csv")

### 파티셔닝
- 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
- 파티셔닝된 디렉터리 또는 테이블에 파일을 쓸 때 디렉터리별로 컬럼 데이터를 인코딩해 저장

In [43]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")
.save("/tmp/partitioned-files.parquet")

### 버켓팅
- 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법
    - 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음
    - 데이터가 이후의 사용 방식에 맞춰 사전에 파티셔닝되므로 조인이나 집계 시 발생하는 고비용을 셔플을 피할 수 있음
- 특정 컬럼을 파티셔닝하면 수억 개의 디렉터리를 만들어낼 수도 있음
    - 이런 경우 데이터를 버켓팅할 수 있는 방법을 찾아야 함

In [44]:
val numberBuckets = 10
val columnToBucketBy = "count"

numberBuckets: Int = 10
columnToBucketBy: String = count


In [46]:
csvFile.write.format("parquet").mode("overwrite")
.bucketBy(numberBuckets, columnToBucketBy).saveAsTable("bucketedFiles")